In [1]:
import requests
from bs4 import BeautifulSoup as soup
import numpy as np
import pandas as pd
import re
import string
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler  


from sklearn.neural_network import MLPRegressor


from sklearn.model_selection import train_test_split



In [2]:
# username: E76723
# password: target01
# login-form-type: pwd

In [3]:

def log_in():
    '''This function logs the scrapper into the main menu with the approiate cookies'''
    login_data = {'username': 'E76723',
                  'password': 'target01',
                 'login-form-type': 'pwd'}

    
    session = requests.Session()
    
    # initial login page 
    
    login_link = "https://www.aksteelonline.com/webapp/wcs/stores/servlet/StoreView?storeId=10001&langId=-1"
    
    login_screen = session.get(login_link)

#     login_soup = soup(login_screen.content)

    # this f value is dynamic

#     login_data['f'] = login_soup.find('input', {'name': 'f'})['value']

   
    
    # form request link
    form_url = 'https://www.aksteelonline.com/pkmslogin.form'

    # sending login data into form
    main_page = session.post(form_url, login_data)
    
    # retreiving catalog page
    catalog_page = session.get(
        'https://www.aksteelonline.com/webapp/wcs/stores/servlet/LogonForm?storeId=10001&catalogId=10001')
    
    return session


In [4]:
session_logged_in = log_in()

In [5]:
# returning and converting BUY_NOW auctions

buy_now_raw = session_logged_in.get('https://www.aksteelonline.com/webapp/wcs/stores/servlet/CatalogSearchResultView?storeId=10001&catalogId=10001&ButtonB=Search+Buy+Now&countPerPage=50&sortSelect=product&page=0').text

buy_now = soup(buy_now_raw, 'html.parser')


In [6]:

# converting the table

buy_now_table = buy_now.find('table', {'border': "0", 'cellpadding':'2'})

In [7]:
# creating list of prices

def price_getter(buy_now_table):

    price = buy_now_table.find_all('td', {'headers': 'WC_CatalogSearchResultDisplay_PriceHeader'})

    price_list = []
    for i in range(len(price)):

        price_list.append(float(re.sub(r'[^\d.]+', '', price[i].text)))
        
    return price_list



In [8]:
# creating list of gauges


def gauge_getter(buy_now_table):
    gauge = buy_now_table.find_all('td', {'headers': 'WC_CatalogSearchResultDisplay_GAHeader'})


    gauge_list = []
    for i in range(len(gauge)):

        gauge_list.append(float(re.sub(r'[^\d.]+', '', gauge[i].text)))
        
    return gauge_list


In [9]:
# creating list of widths

def width_getter(buy_now_table):

    width = buy_now_table.find_all('td', {'headers': 'WC_CatalogSearchResultDisplay_WDHeader'})


    width_list = []
    for i in range(len(width)):

        width_list.append(float(re.sub(r'[^\d.]+', '', width[i].text)))
        
    return width_list

In [10]:
# creating list of weights

def weight_getter(buy_now_table):

    weight = buy_now_table.find_all('td', {'headers': 'WC_CatalogSearchResultDisplay_WeightHeader'})


    weight_list = []
    for i in range(len(weight)):

        weight_list.append(float(re.sub(r'[^\d.]+', '', weight[i].text)))
        
    return weight_list

In [11]:
# creating list of locations


        
def location_getter(buy_now_table):

    location = buy_now_table.find_all('td', {'headers': 'WC_CatalogSearchResultDisplay_LocationHeader'})


    location_list = []
    for i in range(len(location)):

        location_list.append(location[i].text.strip('\r\n\t\t\t\t\t'))
    return location_list
    

In [12]:
# scraping index table
index_table = buy_now_table.find_all('td', {'align':'left'})


In [13]:
# creating index list of unique codes

# table cellpadding="0" cellspacing="0" border="0" width="100%"


def index_getter(index_table):
    index_list = []

    for i in range(len(index_table)):
        index_list.append(index_table[i].text.rstrip().strip('\r\n\t\t\t\t\t\t\t\t\t\xa0\xa0\xa0\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t'))
    return index_list



In [14]:
# scraping inner-table

inner_table = buy_now_table.find_all('table', {'cellpadding' : "0", 'cellspacing' : "0", 'border' : "3",
                                    'width' : "100%"})

In [15]:
# creating list of defects

def defect_getter(inner_table):

    defect_list = []

    for i in range(len(inner_table)):

        if inner_table[i].find(text = re.compile(r"\bDefect\b")) is None:
            defect_list.append(None)
        else:
            defect_list.append(inner_table[i].find(text = re.compile(r"\bDefect\b")).findNext('td').text.rstrip().strip('\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t'))
    return defect_list
#     print(i)

# all_defects = buy_now_table.find_all(text = re.compile(r"\bDefect\b"))    

# buy_now_table.find(text = re.compile(r"\bDefect\b")).findNext('td').text




In [16]:
# creating list of lin-feet

def linfeet_getter(inner_table):
    
    lin_feet_list = []
    for i in range(len(inner_table)):

        if inner_table[i].find(text = re.compile("Lin. Ft.")) is None:
            lin_feet_list.append(None)
        else:
            lin_feet_list.append(float(inner_table[i].find(text = re.compile("Lin. Ft.")).findNext('td').text.rstrip().strip('\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t').replace(",","")))
            
    return lin_feet_list
        
        


In [17]:
# retreiving chemistry data

chem_table = buy_now_table.find_all(text = re.compile("C-"))

def chem_getter(chem_table):

    chem_list = []
    for i in range(len(chem_table)):


        digit_string = re.sub('[^0-9\.]',' ', chem_table[i].rstrip())
        
#         if digit_string is None:
#             chem_list.append(None)
            
#             pass
        

        pretty_string= [float(x) for x in digit_string.split()]

        #logic if code does or does not contain heat number

        
        
        
        if pretty_string[0] > 100:
            pretty_string.pop(0)
        
        if len(pretty_string) != 8:
            chem_list.append(np.repeat(np.nan, 8))
        

        
        else: chem_list.append(pretty_string)
            
            

    return np.array(chem_list)


chem_getter_test = chem_getter(chem_table)


In [18]:
# chem_list = []
# pretty_string_test = []
# for i in range(len(chem_table)):


#     digit_string = re.sub('[^0-9\.]',' ', chem_table[i].rstrip())

#     if digit_string is None:
#         chem_list.append(None)

#         pass


#     pretty_string= [float(x) for x in digit_string.split()]
#     pretty_string_test.append(pretty_string)

# #     #logic if code does or does not contain heat number

# #     if pretty_string[0] > 100:
# #         pretty_string.pop(0)

# #     if pretty_string =! 
    
    
#     chem_list.append(pretty_string)



# digit_string.split()

In [19]:
def url_generator(buy_now):
    url_count = int(buy_now.find('a', {'id' :"WC_PageURL_Link_Next"}).find_previous_sibling('a').text.rstrip().strip('\r\n\t\t\t\t\t'))

    skeleton_url = "https://www.aksteelonline.com/webapp/wcs/stores/servlet/CatalogSearchResultView?storeId=10001&catalogId=10001&ButtonB=Search+Buy+Now&countPerPage=50&sortSelect=product&page="


    page_urls = []
    for i in range(url_count):
        page_urls.append(skeleton_url + str(i))
    return page_urls

In [20]:
def data_getter():

    price_master_list = []
    gauge_master_list = []
    width_master_list = []
    weight_master_list = []
    location_master_list = []
    defect_master_list = []
    linfeet_master_list = []
    index_master_list = []
    
    

    # chemistry lists 
    
    carbon_master = []
    manganese_master = []
    phosphorus_master = []
    sulfur_master = []
    silicon_master = []
    aluminium_master = []
    niobium_master = []
    vanadium_master = []

    
    # logging into session
    session_logged_in = log_in()
    
    # creating input for url generator
    buy_now_raw_url = session_logged_in.get('https://www.aksteelonline.com/webapp/wcs/stores/servlet/CatalogSearchResultView?storeId=10001&catalogId=10001&ButtonB=Search+Buy+Now&countPerPage=50&sortSelect=product&page=0').text

    buy_now_url = soup(buy_now_raw_url, 'html.parser')
    
    
    # generating urls for loop
    page_urls = url_generator(buy_now_url)
    

    for page in page_urls:
        
        # retreiving/updating all required variables for each iteration of loop

        buy_now_raw = session_logged_in.get(page).text

        buy_now = soup(buy_now_raw, 'html.parser')
        
        buy_now_table = buy_now.find('table', {'border': "0", 'cellpadding':'2'})
        
        index_table = buy_now_table.find_all('td', {'align':'left'})
        
        
        inner_table = buy_now_table.find_all('table', {'cellpadding' : "0", 'cellspacing' : "0", 'border' : "3",
                                        'width' : "100%"})
        
        chem_table = buy_now_table.find_all(text = re.compile("C-"))
        
    
        

        # price

        current_price = price_getter(buy_now_table)

        price_master_list.extend(current_price)


        # gauges

        current_gauge = gauge_getter(buy_now_table)

        gauge_master_list.extend(current_gauge)
        

        # widths

        current_width = width_getter(buy_now_table)

        width_master_list.extend(current_width)

        
        # weights

        current_weight = weight_getter(buy_now_table)

        weight_master_list.extend(current_weight)

        
        # locations

        current_location = location_getter(buy_now_table)

        location_master_list.extend(current_location)

        ###


        # defects

        current_defect = defect_getter(inner_table)

        defect_master_list.extend(current_defect)


        # linfeets

        current_linfeet = linfeet_getter(inner_table)

        linfeet_master_list.extend(current_linfeet) 


        ###

        # indexs

        current_index = index_getter(index_table)

        index_master_list.extend(current_index)


        ###


        # pulling all chemicals

        current_chem = chem_getter(chem_table)


        carbon_master.extend(current_chem[:,0].tolist())
        manganese_master.extend(current_chem[:,1].tolist())
        phosphorus_master.extend(current_chem[:,2].tolist())
        sulfur_master.extend(current_chem[:,3].tolist())
        silicon_master.extend(current_chem[:,4].tolist())
        aluminium_master.extend(current_chem[:,5].tolist())
        niobium_master.extend(current_chem[:,6].tolist())
        vanadium_master.extend(current_chem[:,7].tolist())


    comprehensive_dict = {"index" : index_master_list, 'price' : price_master_list ,'carbon' : carbon_master, 'manganese' : manganese_master,
    'phosphorus': phosphorus_master, 'sulfur' : sulfur_master, 'silicon' : silicon_master,
    'aluminium' : aluminium_master, 'niobium' : niobium_master, 'vanadium' : vanadium_master,
    'gauge' : gauge_master_list, 'width' : width_master_list, 'weight' : weight_master_list, 'location' : location_master_list,
    'defect': defect_master_list, "linear feat" : linfeet_master_list}


    ak_dict = pd.DataFrame(comprehensive_dict)

    return ak_dict
    
    
ak_dict = data_getter()

In [22]:
ak_dict.dropna()


,index,price,carbon,manganese,phosphorus,sulfur,silicon,aluminium,niobium,vanadium,gauge,width,weight,location,defect,linear feat
0,681110-02A,30.0,0.0038,0.332,0.015,0.0006,0.300,0.0069,0.005,0.054,0.0572,48.358,38470.0,"AK STEEL CORPORATION Middletown, OH",0606 COATING DENTS,4232.0
1,684481-04BA,30.0,0.0096,0.340,0.023,0.0013,0.390,0.0080,0.017,0.066,0.0470,48.858,9140.0,"AK STEEL CORPORATION Middletown, OH",0933 LITE COIL-QUALITY,1213.0
2,684592-01AA,30.0,0.0100,0.310,0.023,0.0015,0.370,0.0080,0.024,0.065,0.0566,44.582,11850.0,"AK STEEL CORPORATION Middletown, OH",2939 DISTRESSED INV PRV DIV,1426.0
3,684222-04BA,30.0,0.0070,0.348,0.017,0.0006,0.319,0.0071,0.004,0.053,0.0651,47.671,9700.0,"AK STEEL CORPORATION Middletown, OH",0933 LITE COIL-QUALITY,947.0
4,684187-07A,30.0,0.0060,0.335,0.018,0.0011,0.364,0.0084,0.004,0.049,0.0605,38.069,5700.0,"AK STEEL CORPORATION Middletown, OH",0677 AL STAINLESS INTERFACE UNCTD,754.0
5,690705-06B,30.0,0.0046,0.340,0.019,0.0003,0.366,0.0076,0.007,0.058,0.1599,45.580,11850.0,"AK STEEL CORPORATION Rockport, IN",0933 LITE COIL-QUALITY,490.0
6,692827-02AA,30.0,0.0106,0.320,0.023,0.0013,0.410,0.0100,0.018,0.066,0.0657,45.158,14870.0,"AK STEEL CORPORATION Middletown, OH",0602 CTG BUILDUP FLANGE,1510.0
7,692475-07BA,30.0,0.0047,0.287,0.017,0.0009,0.301,0.0070,0.004,0.052,0.0415,33.704,10540.0,"AK STEEL CORPORATION Middletown, OH",0677 AL STAINLESS INTERFACE UNCTD,2296.0
8,700795-01A,30.0,0.0084,0.320,0.024,0.0025,0.360,0.0080,0.021,0.067,0.0233,34.604,7090.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0665 CTG CL KNIFE MRKS,2683.0
9,700734-03AA,30.0,0.0091,0.240,0.022,0.0032,0.370,0.0080,0.006,0.056,0.0469,46.958,15010.0,"AK STEEL CORPORATION Middletown, OH",0602 CTG BUILDUP FLANGE,2071.0


In [22]:
# cleaning df 

ak_dict['defect_code'] = ak_dict.defect.str.extract('(\d+)')

ak_dict_clean = ak_dict.dropna()



In [23]:
ak_dict_clean

,index,price,carbon,manganese,phosphorus,sulfur,silicon,aluminium,niobium,vanadium,gauge,width,weight,location,defect,linear feat,defect_code
0,684778-06AA,30.0,0.0038,0.332,0.015,0.0006,0.300,0.0069,0.005,0.054,0.0411,45.347,21350.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0616 COATING SCRATCHES,3512.0,0616
1,681110-02A,30.0,0.0038,0.332,0.015,0.0006,0.300,0.0069,0.005,0.054,0.0572,48.358,38470.0,"AK STEEL CORPORATION Middletown, OH",0606 COATING DENTS,4232.0,0606
2,686359-01,30.0,0.0052,0.431,0.016,0.0006,0.317,0.0074,0.001,0.058,0.0370,48.000,36870.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0636 COATING DROSS,5774.0,0636
3,684481-04BA,30.0,0.0096,0.340,0.023,0.0013,0.390,0.0080,0.017,0.066,0.0470,48.858,9140.0,"AK STEEL CORPORATION Middletown, OH",0933 LITE COIL-QUALITY,1213.0,0933
4,684592-01AA,30.0,0.0100,0.310,0.023,0.0015,0.370,0.0080,0.024,0.065,0.0566,44.582,11850.0,"AK STEEL CORPORATION Middletown, OH",2939 DISTRESSED INV PRV DIV,1426.0,2939
5,664153-03AA,30.0,0.0060,0.430,0.018,0.0010,0.324,0.0077,0.011,0.067,0.0480,22.500,5289.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0675 CTG SLOW RUN,1418.0,0675
6,664153-03AB,30.0,0.0060,0.430,0.018,0.0010,0.324,0.0077,0.011,0.067,0.0480,22.500,5292.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0675 CTG SLOW RUN,1418.0,0675
7,684222-04BA,30.0,0.0070,0.348,0.017,0.0006,0.319,0.0071,0.004,0.053,0.0651,47.671,9700.0,"AK STEEL CORPORATION Middletown, OH",0933 LITE COIL-QUALITY,947.0,0933
8,684187-07A,30.0,0.0060,0.335,0.018,0.0011,0.364,0.0084,0.004,0.049,0.0605,38.069,5700.0,"AK STEEL CORPORATION Middletown, OH",0677 AL STAINLESS INTERFACE UNCTD,754.0,0677
9,690705-06B,30.0,0.0046,0.340,0.019,0.0003,0.366,0.0076,0.007,0.058,0.1599,45.580,11850.0,"AK STEEL CORPORATION Rockport, IN",0933 LITE COIL-QUALITY,490.0,0933


In [24]:
# saving_clean df to working directory
ak_dict_clean.to_csv('clean_ak_df',index = False)



In [25]:
# splitting data into test and train

train, test = train_test_split(ak_dict_clean)

In [26]:
# creating train and test 

X_train = train.loc[:, ['carbon', 'manganese', 'phosphorus',
              'sulfur', 'silicon', 'aluminium', 'niobium',
              'vanadium', 'gauge', 'width',
              'weight', 'linear feat']].values

y_train = train.loc[:, "price"].values

X_test = test.loc[:, ['carbon', 'manganese', 'phosphorus',
              'sulfur', 'silicon', 'aluminium', 'niobium',
              'vanadium', 'gauge', 'width',
              'weight', 'linear feat']].values

# 'defect_code'

y_test = test.loc[:, "price"].values




# standardizing input data for mlp regression 

x_scaler = StandardScaler()  
x_scaler.fit(X_train)  

X_train = x_scaler.transform(X_train)  

# apply same transformation to test data
X_test = x_scaler.transform(X_test)  



# standardizing output data for mlp regression 

y_train = y_train.reshape(-1,1)

y_scaler = StandardScaler()  
y_scaler.fit(y_train)  

y_train = y_scaler.transform(y_train)  

# apply same transformation to test data

y_test = y_test.reshape(-1,1)


y_test = y_scaler.transform(y_test)

In [27]:
ak_regressor = MLPRegressor()

ak_regressor.fit(X_train, np.ravel(y_train))

ak_regressor.score(X_test, np.ravel(y_test))

/Users/austinader/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.6584657244681804

In [1]:


y_scaler.inverse_transform(ak_regressor.predict(X_test[2].reshape(1,-1)))

NameError: name 'y_scaler' is not defined

In [29]:
test

,index,price,carbon,manganese,phosphorus,sulfur,silicon,aluminium,niobium,vanadium,gauge,width,weight,location,defect,linear feat,defect_code
44,678460-01AD,65.0,0.0093,0.330,0.029,0.0017,0.410,0.0290,0.570,0.113,0.0461,12.500,2977.0,"PRECISION STRIP (MIDDLETOWN) Middletown, OH",0929 UNIDENTFD ROLL MARK,1549.0,0929
147,SKD355596,55.0,0.0080,0.276,0.028,0.0013,0.490,0.0053,0.561,0.064,0.0190,17.625,2923.0,"AK STEEL CORPORATION Coshocton, OH",0397 COLD MILL CHATTER,2564.0,0397
279,671920-03AAB,62.0,0.0109,0.270,0.026,0.0011,0.530,0.0130,0.010,0.089,0.0669,14.000,10777.0,"PRECISION STRIP WOODBURN Woodburn, KY",0023 MELT BLACK LINES,3380.0,0023
138,SKD349760,55.0,0.0130,0.369,0.028,0.0010,0.421,0.0038,0.553,0.079,0.0190,13.250,3320.0,"AK STEEL CORPORATION Coshocton, OH",0396 COLD MILL RUMBLE CHATTER,3874.0,0396
288,687519-03AB,62.0,0.0072,0.300,0.024,0.0009,0.430,0.0120,0.021,0.096,0.0847,14.083,10592.0,"PRECISION STRIP WOODBURN Woodburn, KY",0023 MELT BLACK LINES,2609.0,0023
164,SKD360468,55.0,0.0120,0.408,0.029,0.0019,0.376,0.0044,0.550,0.068,0.0190,18.250,5756.0,"AK STEEL CORPORATION Coshocton, OH",0564 TM/CRL SCRATCH,4877.0,0564
258,687365-02BB,62.0,0.0144,0.320,0.024,0.0011,0.350,0.0090,0.024,0.096,0.0573,16.000,12525.0,"PRECISION STRIP, INC.(MINSTER) Minster, OH",0023 MELT BLACK LINES,4013.0,0023
39,658514-01AB,65.0,0.0095,0.320,0.025,0.0009,0.430,0.0260,0.590,0.112,0.0716,29.850,19980.0,"PRECISION STRIP, INC.(MINSTER) Minster, OH",2914 PRIME ORDER CANCELLED,2833.0,2914
111,685996-04AA,47.0,0.0064,0.383,0.019,0.0017,0.304,0.0069,0.004,0.055,0.0896,61.170,45840.0,"AK STEEL CORPORATION Rockport, IN",2961 PROC/PROD TRIAL,2524.0,2961
180,SKD359949,61.0,0.0190,0.308,0.026,0.0024,0.524,0.0049,0.449,0.071,0.0190,37.250,12820.0,"AK STEEL CORPORATION Coshocton, OH",0174 HSM WORN ROLL PATTERN,5321.0,0174
